In [76]:
# minimize criterias:
# J_v = 1 / Ki
# J_u = K_inf
# M_s = max(abs(S(jw))), M_s <= 1.7
# M_t = max(abs(T(jw))), M_t <= 1.3
# S(jw) = 1 / (1 + L(jw))
# T(jw) = L(jw) / (1 + L(jw))

import numpy as np

def differentiate(f, x, h=0.01):
    return (f(x + h) - f(x - h)) / (2 * h)

def get_gradient(f, x, h=0.01):
    gradient = []
    for i in range(len(x)):
        xi1, xi2 = x.copy(), x.copy()
        xi1[i] += h
        xi2[i] -= h
        gradient.append((f(xi1) - f(xi2)) / (2 * h))
    return np.array(gradient)

def gradient_descent(x0, f, gamma, maximize=False, h=0.01, epsilon=1e-15, iterations=1e5, verbose=False):
    i = 0
    x1 = x0
    while True:
        gradient = get_gradient(f, x0, h) * (-1 if maximize else 1)
        x1, x0 = x0 - gamma * gradient, x1
        i += 1
        
        if verbose:
            if x1[0] < Wc_min:
                x1[0] = Wc_min
            for i in range(1,len(x1)):
                if x1[i] < 0.1:
                    x1[i] = 0.1
            
            print(f"i: {i}, x = {x0}")
            print(f"i: {i}, gradient = {gradient}")
            print(f"i: {i}, x' = {x1}")
            f(x1, verbose=True)

        if abs(sum(gradient**2)**0.5) < epsilon or i >= iterations:
            return x1

def criteria_function(x, verbose=False):
    Wc = x[0]
    tau = x[1]
    beta = x[2]
    zeta = x[3]

    def G_abs(W):
        return Kg / (W * (1 + (Tg*W)**2)**0.5)

    K_inf = Wc * tau * beta * (1 + (Wc*tau/beta)**2)**0.5 / (G_abs(Wc) * (((1 - (Wc*tau)**2)**2 ) + (2*zeta*Wc*tau)**2)**0.5 )
    Ki = K_inf / (tau * beta)

    def F_abs(W):
        return Ki * (( (1-(W*tau)**2)**2 + (2*zeta*W*tau)**2 )**0.5) / (W * (1 + (W*tau/beta)**2)**0.5)

    def S_abs(W):
        W = W[0]
        return 1 / (1 + F_abs(W) * G_abs(W))

    def T_abs(W):
        W = W[0]
        return F_abs(W) * G_abs(W) / (1 + F_abs(W) * G_abs(W))

    def FS_abs(W):
        return F_abs(W[0]) * S_abs(W)

    def SW_abs(W):
        return S_abs(W) / W[0]

    J_v = SW_abs(gradient_descent([1], SW_abs, Wc, maximize=True, epsilon=1e-6, iterations=1000, verbose=False))
    J_u = FS_abs(gradient_descent([1], FS_abs, Wc, maximize=True, epsilon=1e-6, iterations=1000, verbose=False))
    M_s = S_abs(gradient_descent([1], S_abs, Wc, maximize=True, epsilon=1e-6, iterations=1000, verbose=False))
    M_t = T_abs(gradient_descent([1], T_abs, Wc, maximize=True, epsilon=1e-6, iterations=1000, verbose=False))

    J_tot = (J_v**2 + J_u**2)**0.5
    if verbose:
        print(f"J_tot = {round(J_tot, 5)}, J_v = {round(J_v, 5)}, J_u = {round(J_u, 5)}, M_s = {round(M_s, 5)}, M_t = {round(M_t, 5)}")
    
    return J_tot

Ra = 0.11
Ke = 0.022  
Jm = 8.7e-5
Bm = 8.7e-5
N_torque = 50
N_speed = 1 / N_torque
Kg = N_speed * Ke / (Ra * Bm + Ke**2)
Tg = (Ra * Jm) / (Ra * Bm + Ke**2)
W_G150 = (3**0.5) / Tg

Wc_min = 0.3 * W_G150
Wc_max = 0.5 * W_G150

x0 = [10, 10, 10, 10]
print("min at x =", gradient_descent(x0, criteria_function, 10, epsilon=1e-14, verbose=True, iterations=10000))

i: 3, x = [10, 10, 10, 10]
i: 3, gradient = [ 1.17878601  0.03071957  0.01071892 -0.04164665]
i: 3, x' = [26.79900681  9.69280429  9.89281078 10.41646647]
J_tot = 33.06792, J_v = 0.02708, J_u = 33.06791, M_s = 0.84641, M_t = 0.99645
i: 3, x = [26.79900681  9.69280429  9.89281078 10.41646647]
i: 3, gradient = [ 1.17878601  0.03071957  0.01071892 -0.04164665]
i: 3, x' = [26.79900681  9.69280429  9.89281078 10.41646647]
J_tot = 33.06792, J_v = 0.02708, J_u = 33.06791, M_s = 0.84641, M_t = 0.99645
i: 3, x = [26.79900681  9.69280429  9.89281078 10.41646647]
i: 3, gradient = [ 1.51696164  0.01668233  0.00481659 -0.02019082]
i: 3, x' = [26.79900681  9.52598103  9.84464485 10.61837466]
J_tot = 33.06057, J_v = 0.02695, J_u = 33.06056, M_s = 0.84645, M_t = 0.99663
i: 3, x = [26.79900681  9.52598103  9.84464485 10.61837466]
i: 3, gradient = [ 1.51696164  0.01668233  0.00481659 -0.02019082]
i: 3, x' = [26.79900681  9.52598103  9.84464485 10.61837466]
J_tot = 33.06057, J_v = 0.02695, J_u = 33.06056